In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import subprocess
import shutil
import time
import pandas as pd
import datetime

import requests
import lxml
from lxml.html import fromstring

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import pyperclip

# headers
userAgent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36'
headers = {'User-Agent' : userAgent}

# option 
option = Options()
option.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

In [9]:
# 로그인을 위한 함수 구현
def clipboard_input(user_xpath, user_input):
    temp_user_input = pyperclip.paste()  # 사용자 클립보드를 따로 저장

    pyperclip.copy(user_input)
    driver.find_element_by_xpath(user_xpath).click()
    ActionChains(driver).key_down(Keys.CONTROL).send_keys('v').key_up(Keys.CONTROL).perform()

    pyperclip.copy(temp_user_input)  # 사용자 클립보드에 저장 된 내용을 다시 가져 옴
    time.sleep(1)

# 파라미터


In [10]:
ID = ''
PASSWORD = ''

# 밴드 사이트에 접근

In [12]:
# chrome.exe로 접근
subprocess.Popen(r'C:\Program Files\Google\Chrome\Application\chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\chrometemp"') # 디버거 크롬 구동

# url 접근
driver = webdriver.Chrome('90/chromedriver.exe', options=option)
url = 'https://band.us/'
driver.get(url)
time.sleep(2)

# 로그인 클릭
driver.find_element_by_xpath('//*[@id="header"]/div/div/a[2]').click()
time.sleep(2)

# 네이버로 로그인 클릭
driver.find_element_by_xpath('//*[@id="login_list"]/li[3]/a/span').click()

# ID, PW 넣고 로그인
login = {
    "id" : ID,
    "pw" : PASSWORD
}

clipboard_input('//*[@id="id"]', login.get("id"))
clipboard_input('//*[@id="pw"]', login.get("pw"))

# 로그인 상태 유지 클릭
driver.find_element_by_xpath('//*[@id="label_login_chk"]').click()

# 로그인하기 클릭
driver.find_element_by_xpath('//*[@id="log.login"]').click()
time.sleep(2)

# 특정 밴드 클릭
driver.find_element_by_xpath('//*[@id="content"]/div/section/div[2]/div/div/ul/li[2]/div/div/a/div[2]/p').click()
time.sleep(2)

# 보기방식 변경
driver.find_element_by_xpath('//*[@id="content"]/section/div[13]/div[1]/div/button[2]').click()
time.sleep(2)

# 정보 크롤링
## 데이터프레임 초기화 주의!!

In [ ]:
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [22]:
# 정보수집
num = 516
#df = pd.DataFrame(columns = ['작성자', '일자', '좋아요개수', '댓글개수', '조회수', '내용'])
t1 = time.time()

while True:
    
    # 정보수집 
    
    
    ## 작성자
    try:
        writer = driver.find_element_by_xpath(f'//*[@id="content"]/section/div[14]/div[2]/div/div[{num}]/div/div[2]/div/span/strong').text
    
    ### Stopping Condition: 작성자가 수집이 안되었을때 이는 게시글이 없는 것으로 판단
    except:
        
        try:
            print('== 1차오류 발생 == 스크롤을 추가로 내려봅니다.')
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            writer = driver.find_element_by_xpath(f'//*[@id="content"]/section/div[14]/div[2]/div/div[{num}]/div/div[2]/div/span/strong').text
            
        except:
            print(f'정보가 모두 수집되었거나 오류가 발생하였습니다. == 마지막 게시글의 작성일자: {date}')
            break
        
        
    ## 일자   
    date = driver.find_element_by_xpath(f'//*[@id="content"]/section/div[14]/div[2]/div/div[{num}]/div/div[2]/div/div/a/time').text
    
    if ('시간' in date) | ('분' in date): ### 20시간 전 등의 데이터에 대해서는 오늘 날짜를 기입
        today = datetime.datetime.today()
        date = '%s년 %s월 %s일' % ("{:d}".format(today.year), "{:d}".format(today.month), "{:d}".format(today.day))

        
    ## 좋아요 개수
    like_count = driver.find_element_by_xpath(f'//*[@id="content"]/section/div[14]/div[2]/div/div[{num}]/div/div[4]/div[1]/div[1]/button/div/span[2]').text
    
    
    ## 댓글 개수
    comment_count = driver.find_element_by_xpath(f'//*[@id="content"]/section/div[14]/div[2]/div/div[{num}]/div/div[4]/div[1]/div[2]/a/span[2]').text
    
    
    ## 조회수 개수
    try:
        view_count = driver.find_element_by_xpath(f'//*[@id="content"]/section/div[14]/div[2]/div/div[{num}]/div/div[4]/div[2]/span/span[2]').text
    
    except: ### 조회수가 없으면 해당 element가 활성화 안되어 오류가 발생
        view_count = 0
        
        
    ## 내용
    content = driver.find_element_by_xpath(f'//*[@id="content"]/section/div[14]/div[2]/div/div[{num}]/div/div[3]/div/div/div[1]/p').text

    
    
    
    # 데이터프레임에 적재
    tmp = pd.DataFrame({'작성자':[writer],
                     '일자':[date],
                     '좋아요개수':[like_count],
                     '댓글개수':[comment_count],
                     '조회수':[view_count],
                     '내용':[content]})

    df = df.append(tmp, ignore_index = True)
    
    # scroll down(주기: 5)
    if num % 5 == 0:
        print(f'== {num}개 게시글 수집 완료 == Scroll Down!! == 경과시간: {round(time.time() - t1)}초')
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        
    
    # num 추가
    num += 1

== 520개 게시글 수집 완료 == Scroll Down!! == 경과시간: 2초
== 525개 게시글 수집 완료 == Scroll Down!! == 경과시간: 4초
== 530개 게시글 수집 완료 == Scroll Down!! == 경과시간: 7초
== 535개 게시글 수집 완료 == Scroll Down!! == 경과시간: 10초
== 540개 게시글 수집 완료 == Scroll Down!! == 경과시간: 13초
== 545개 게시글 수집 완료 == Scroll Down!! == 경과시간: 16초
== 550개 게시글 수집 완료 == Scroll Down!! == 경과시간: 19초
== 555개 게시글 수집 완료 == Scroll Down!! == 경과시간: 22초
== 560개 게시글 수집 완료 == Scroll Down!! == 경과시간: 25초
== 565개 게시글 수집 완료 == Scroll Down!! == 경과시간: 28초
== 570개 게시글 수집 완료 == Scroll Down!! == 경과시간: 31초
== 575개 게시글 수집 완료 == Scroll Down!! == 경과시간: 33초
== 580개 게시글 수집 완료 == Scroll Down!! == 경과시간: 36초
== 585개 게시글 수집 완료 == Scroll Down!! == 경과시간: 38초
== 590개 게시글 수집 완료 == Scroll Down!! == 경과시간: 41초
== 595개 게시글 수집 완료 == Scroll Down!! == 경과시간: 43초
== 600개 게시글 수집 완료 == Scroll Down!! == 경과시간: 46초
== 605개 게시글 수집 완료 == Scroll Down!! == 경과시간: 48초
== 610개 게시글 수집 완료 == Scroll Down!! == 경과시간: 51초
== 615개 게시글 수집 완료 == Scroll Down!! == 경과시간: 53초
== 620개 게시글 수집 완료 == Scroll Down!! == 경과시간:

== 1355개 게시글 수집 완료 == Scroll Down!! == 경과시간: 518초
== 1360개 게시글 수집 완료 == Scroll Down!! == 경과시간: 522초
== 1365개 게시글 수집 완료 == Scroll Down!! == 경과시간: 525초
== 1370개 게시글 수집 완료 == Scroll Down!! == 경과시간: 529초
== 1375개 게시글 수집 완료 == Scroll Down!! == 경과시간: 533초
== 1380개 게시글 수집 완료 == Scroll Down!! == 경과시간: 537초
== 1385개 게시글 수집 완료 == Scroll Down!! == 경과시간: 540초
== 1390개 게시글 수집 완료 == Scroll Down!! == 경과시간: 544초
== 1395개 게시글 수집 완료 == Scroll Down!! == 경과시간: 548초
== 1400개 게시글 수집 완료 == Scroll Down!! == 경과시간: 552초
== 1405개 게시글 수집 완료 == Scroll Down!! == 경과시간: 555초
== 1410개 게시글 수집 완료 == Scroll Down!! == 경과시간: 559초
== 1415개 게시글 수집 완료 == Scroll Down!! == 경과시간: 563초
== 1420개 게시글 수집 완료 == Scroll Down!! == 경과시간: 567초
== 1425개 게시글 수집 완료 == Scroll Down!! == 경과시간: 571초
== 1430개 게시글 수집 완료 == Scroll Down!! == 경과시간: 575초
== 1435개 게시글 수집 완료 == Scroll Down!! == 경과시간: 579초
== 1440개 게시글 수집 완료 == Scroll Down!! == 경과시간: 582초
== 1445개 게시글 수집 완료 == Scroll Down!! == 경과시간: 586초
== 1450개 게시글 수집 완료 == Scroll Down!! == 경과시간: 590초


== 2170개 게시글 수집 완료 == Scroll Down!! == 경과시간: 1245초
== 2175개 게시글 수집 완료 == Scroll Down!! == 경과시간: 1250초
== 2180개 게시글 수집 완료 == Scroll Down!! == 경과시간: 1255초
== 2185개 게시글 수집 완료 == Scroll Down!! == 경과시간: 1260초
== 2190개 게시글 수집 완료 == Scroll Down!! == 경과시간: 1265초
== 2195개 게시글 수집 완료 == Scroll Down!! == 경과시간: 1271초
== 2200개 게시글 수집 완료 == Scroll Down!! == 경과시간: 1276초
== 2205개 게시글 수집 완료 == Scroll Down!! == 경과시간: 1281초
== 2210개 게시글 수집 완료 == Scroll Down!! == 경과시간: 1286초
== 2215개 게시글 수집 완료 == Scroll Down!! == 경과시간: 1292초
== 2220개 게시글 수집 완료 == Scroll Down!! == 경과시간: 1297초
== 2225개 게시글 수집 완료 == Scroll Down!! == 경과시간: 1305초
== 2230개 게시글 수집 완료 == Scroll Down!! == 경과시간: 1310초
== 2235개 게시글 수집 완료 == Scroll Down!! == 경과시간: 1316초
== 2240개 게시글 수집 완료 == Scroll Down!! == 경과시간: 1321초
== 2245개 게시글 수집 완료 == Scroll Down!! == 경과시간: 1326초
== 2250개 게시글 수집 완료 == Scroll Down!! == 경과시간: 1332초
== 2255개 게시글 수집 완료 == Scroll Down!! == 경과시간: 1337초
== 2260개 게시글 수집 완료 == Scroll Down!! == 경과시간: 1342초
== 2265개 게시글 수집 완료 == Scroll Do

== 2975개 게시글 수집 완료 == Scroll Down!! == 경과시간: 2234초
== 2980개 게시글 수집 완료 == Scroll Down!! == 경과시간: 2241초
== 2985개 게시글 수집 완료 == Scroll Down!! == 경과시간: 2248초
== 2990개 게시글 수집 완료 == Scroll Down!! == 경과시간: 2254초
== 2995개 게시글 수집 완료 == Scroll Down!! == 경과시간: 2261초
== 3000개 게시글 수집 완료 == Scroll Down!! == 경과시간: 2268초
== 3005개 게시글 수집 완료 == Scroll Down!! == 경과시간: 2275초
== 3010개 게시글 수집 완료 == Scroll Down!! == 경과시간: 2281초
== 3015개 게시글 수집 완료 == Scroll Down!! == 경과시간: 2288초
== 3020개 게시글 수집 완료 == Scroll Down!! == 경과시간: 2294초
== 3025개 게시글 수집 완료 == Scroll Down!! == 경과시간: 2301초
== 3030개 게시글 수집 완료 == Scroll Down!! == 경과시간: 2308초
== 3035개 게시글 수집 완료 == Scroll Down!! == 경과시간: 2315초
== 3040개 게시글 수집 완료 == Scroll Down!! == 경과시간: 2322초
== 3045개 게시글 수집 완료 == Scroll Down!! == 경과시간: 2328초
== 3050개 게시글 수집 완료 == Scroll Down!! == 경과시간: 2335초
== 3055개 게시글 수집 완료 == Scroll Down!! == 경과시간: 2342초
== 3060개 게시글 수집 완료 == Scroll Down!! == 경과시간: 2349초
== 3065개 게시글 수집 완료 == Scroll Down!! == 경과시간: 2357초
== 3070개 게시글 수집 완료 == Scroll Do

== 3780개 게시글 수집 완료 == Scroll Down!! == 경과시간: 3426초
== 3785개 게시글 수집 완료 == Scroll Down!! == 경과시간: 3433초
== 3790개 게시글 수집 완료 == Scroll Down!! == 경과시간: 3442초
== 3795개 게시글 수집 완료 == Scroll Down!! == 경과시간: 3450초
== 3800개 게시글 수집 완료 == Scroll Down!! == 경과시간: 3459초
== 3805개 게시글 수집 완료 == Scroll Down!! == 경과시간: 3466초
== 3810개 게시글 수집 완료 == Scroll Down!! == 경과시간: 3474초
== 3815개 게시글 수집 완료 == Scroll Down!! == 경과시간: 3482초
== 3820개 게시글 수집 완료 == Scroll Down!! == 경과시간: 3491초
== 3825개 게시글 수집 완료 == Scroll Down!! == 경과시간: 3499초
== 3830개 게시글 수집 완료 == Scroll Down!! == 경과시간: 3507초
== 3835개 게시글 수집 완료 == Scroll Down!! == 경과시간: 3514초
== 3840개 게시글 수집 완료 == Scroll Down!! == 경과시간: 3523초
== 3845개 게시글 수집 완료 == Scroll Down!! == 경과시간: 3531초
== 3850개 게시글 수집 완료 == Scroll Down!! == 경과시간: 3539초
== 3855개 게시글 수집 완료 == Scroll Down!! == 경과시간: 3547초
== 3860개 게시글 수집 완료 == Scroll Down!! == 경과시간: 3555초
== 3865개 게시글 수집 완료 == Scroll Down!! == 경과시간: 3563초
== 3870개 게시글 수집 완료 == Scroll Down!! == 경과시간: 3572초
== 3875개 게시글 수집 완료 == Scroll Do

== 4585개 게시글 수집 완료 == Scroll Down!! == 경과시간: 4968초
== 4590개 게시글 수집 완료 == Scroll Down!! == 경과시간: 4978초
== 4595개 게시글 수집 완료 == Scroll Down!! == 경과시간: 4986초
== 4600개 게시글 수집 완료 == Scroll Down!! == 경과시간: 4996초
== 4605개 게시글 수집 완료 == Scroll Down!! == 경과시간: 5006초
== 4610개 게시글 수집 완료 == Scroll Down!! == 경과시간: 5016초
== 4615개 게시글 수집 완료 == Scroll Down!! == 경과시간: 5024초
== 4620개 게시글 수집 완료 == Scroll Down!! == 경과시간: 5035초
== 4625개 게시글 수집 완료 == Scroll Down!! == 경과시간: 5044초
== 4630개 게시글 수집 완료 == Scroll Down!! == 경과시간: 5054초
== 4635개 게시글 수집 완료 == Scroll Down!! == 경과시간: 5063초
== 4640개 게시글 수집 완료 == Scroll Down!! == 경과시간: 5073초
== 4645개 게시글 수집 완료 == Scroll Down!! == 경과시간: 5082초
== 4650개 게시글 수집 완료 == Scroll Down!! == 경과시간: 5092초
== 4655개 게시글 수집 완료 == Scroll Down!! == 경과시간: 5102초
== 4660개 게시글 수집 완료 == Scroll Down!! == 경과시간: 5111초
== 4665개 게시글 수집 완료 == Scroll Down!! == 경과시간: 5121초
== 4670개 게시글 수집 완료 == Scroll Down!! == 경과시간: 5131초
== 4675개 게시글 수집 완료 == Scroll Down!! == 경과시간: 5140초
== 4680개 게시글 수집 완료 == Scroll Do

== 5390개 게시글 수집 완료 == Scroll Down!! == 경과시간: 6642초
== 5395개 게시글 수집 완료 == Scroll Down!! == 경과시간: 6655초
== 5400개 게시글 수집 완료 == Scroll Down!! == 경과시간: 6666초
== 5405개 게시글 수집 완료 == Scroll Down!! == 경과시간: 6678초
== 5410개 게시글 수집 완료 == Scroll Down!! == 경과시간: 6689초
== 5415개 게시글 수집 완료 == Scroll Down!! == 경과시간: 6700초
== 5420개 게시글 수집 완료 == Scroll Down!! == 경과시간: 6711초
== 5425개 게시글 수집 완료 == Scroll Down!! == 경과시간: 6722초
== 5430개 게시글 수집 완료 == Scroll Down!! == 경과시간: 6733초
== 5435개 게시글 수집 완료 == Scroll Down!! == 경과시간: 6743초
== 5440개 게시글 수집 완료 == Scroll Down!! == 경과시간: 6755초
== 5445개 게시글 수집 완료 == Scroll Down!! == 경과시간: 6766초
== 5450개 게시글 수집 완료 == Scroll Down!! == 경과시간: 6777초
== 5455개 게시글 수집 완료 == Scroll Down!! == 경과시간: 6787초
== 5460개 게시글 수집 완료 == Scroll Down!! == 경과시간: 6799초
== 5465개 게시글 수집 완료 == Scroll Down!! == 경과시간: 6810초
== 5470개 게시글 수집 완료 == Scroll Down!! == 경과시간: 6819초
== 5475개 게시글 수집 완료 == Scroll Down!! == 경과시간: 6831초
== 5480개 게시글 수집 완료 == Scroll Down!! == 경과시간: 6843초
== 5485개 게시글 수집 완료 == Scroll Do

== 6195개 게시글 수집 완료 == Scroll Down!! == 경과시간: 8528초
== 6200개 게시글 수집 완료 == Scroll Down!! == 경과시간: 8539초
== 6205개 게시글 수집 완료 == Scroll Down!! == 경과시간: 8551초
== 6210개 게시글 수집 완료 == Scroll Down!! == 경과시간: 8564초
== 6215개 게시글 수집 완료 == Scroll Down!! == 경과시간: 8576초
== 6220개 게시글 수집 완료 == Scroll Down!! == 경과시간: 8588초
== 6225개 게시글 수집 완료 == Scroll Down!! == 경과시간: 8601초
== 6230개 게시글 수집 완료 == Scroll Down!! == 경과시간: 8613초
== 6235개 게시글 수집 완료 == Scroll Down!! == 경과시간: 8625초
== 6240개 게시글 수집 완료 == Scroll Down!! == 경과시간: 8638초
== 6245개 게시글 수집 완료 == Scroll Down!! == 경과시간: 8650초
== 6250개 게시글 수집 완료 == Scroll Down!! == 경과시간: 8663초
== 6255개 게시글 수집 완료 == Scroll Down!! == 경과시간: 8674초
== 6260개 게시글 수집 완료 == Scroll Down!! == 경과시간: 8688초
== 6265개 게시글 수집 완료 == Scroll Down!! == 경과시간: 8700초
== 6270개 게시글 수집 완료 == Scroll Down!! == 경과시간: 8713초
== 6275개 게시글 수집 완료 == Scroll Down!! == 경과시간: 8724초
== 6280개 게시글 수집 완료 == Scroll Down!! == 경과시간: 8737초
== 6285개 게시글 수집 완료 == Scroll Down!! == 경과시간: 8750초
== 6290개 게시글 수집 완료 == Scroll Do

== 6995개 게시글 수집 완료 == Scroll Down!! == 경과시간: 10665초
== 7000개 게시글 수집 완료 == Scroll Down!! == 경과시간: 10680초
== 7005개 게시글 수집 완료 == Scroll Down!! == 경과시간: 10694초
== 7010개 게시글 수집 완료 == Scroll Down!! == 경과시간: 10709초
== 7015개 게시글 수집 완료 == Scroll Down!! == 경과시간: 10722초
== 7020개 게시글 수집 완료 == Scroll Down!! == 경과시간: 10737초
== 7025개 게시글 수집 완료 == Scroll Down!! == 경과시간: 10751초
== 7030개 게시글 수집 완료 == Scroll Down!! == 경과시간: 10765초
== 7035개 게시글 수집 완료 == Scroll Down!! == 경과시간: 10779초
== 7040개 게시글 수집 완료 == Scroll Down!! == 경과시간: 10793초
== 7045개 게시글 수집 완료 == Scroll Down!! == 경과시간: 10807초
== 7050개 게시글 수집 완료 == Scroll Down!! == 경과시간: 10821초
== 7055개 게시글 수집 완료 == Scroll Down!! == 경과시간: 10835초
== 7060개 게시글 수집 완료 == Scroll Down!! == 경과시간: 10850초
== 7065개 게시글 수집 완료 == Scroll Down!! == 경과시간: 10865초
== 7070개 게시글 수집 완료 == Scroll Down!! == 경과시간: 10878초
== 7075개 게시글 수집 완료 == Scroll Down!! == 경과시간: 10891초
== 7080개 게시글 수집 완료 == Scroll Down!! == 경과시간: 10907초
== 7085개 게시글 수집 완료 == Scroll Down!! == 경과시간: 10921초
== 7090개 게시글

WebDriverException: Message: disconnected: received Inspector.detached event
  (Session info: chrome=91.0.4472.77)


In [23]:
df

,작성자,일자,좋아요개수,댓글개수,조회수,내용
0,이시우,25분 전,1,0,11,"한국어로 듣는 모차르트 얘기, '꽃보다 할배'가 놀란 풍경\nhttps://news..."
1,이진억(산머루),2021년 6월 4일,5,1,30,"#국내산행소식/정보 \n♡영남알프스9봉,천황산(사자봉)\n 재약산(수미봉)♡..."
2,정강호 67 (빛고을),2021년 6월 4일,7,3,20,#여행후기_mr.j 146\n#전남영암 ('21. 5. 9)\n#영암풍력발전\n#광...
3,이점수,2021년 6월 4일,18,8,99,21년06월03일 목요일 아침산책.\n접시꽃 당신!\n비오는날에....!
4,정강호 67 (빛고을),2021년 6월 3일 오전 9:40,18,16,101,#여행후기_mr.j 145\n#전남화순 2 ('21. 5.19)\n#화순풍력발전\n...
...,...,...,...,...,...,...
7571,임태이,2018년 10월 29일 오후 6:19,19,12,243,북한산 ~~ 숨은벽 단풍산행
7572,신령(양선태),2018년 10월 29일 오후 6:10,13,5,117,광주 무등산 중봉 ~ 화순 이서 OK목장 까지 달리다.\n2018년 10월 27일(...
7573,신어산(동김해산악회),2018년 10월 29일 오후 6:05,13,5,107,단양 도락산행!!\n18년 10월 28일!!
7574,박기수,2018년 10월 29일 오후 1:45,15,8,190,2018.10.27(토) KOIMA성무산우회\n울진.금강소나무숲길 4구간(600년 ...


In [19]:
num

516

In [21]:
writer = driver.find_element_by_xpath(f'//*[@id="content"]/section/div[14]/div[2]/div/div[{num}]/div/div[2]/div/span/strong').text
writer

'김천규 서울 관악구 법원단지 1948'

In [ ]:
#df.to_excel('../데이터/네이버_밴드_크롤링.xlsx', index=False)